In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
top_60_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(60)
top_60_coins_df = top_60_coins_df.reset_index(drop=True)
top_60_coins = top_60_coins_df.get('name')

In [3]:
from constants.coin_names import CoinNames
from settings import DATA_DIR
import pandas as pd
import os
dfs = {}
no_data = []
# for coin in CoinNames.__members__:
for coin in top_60_coins:
    p = DATA_DIR / f'{coin}.csv'
    if os.path.exists(p):
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        dfs[coin] = df
    else:
        no_data.append(coin)
else:
    os.sys.stderr.write(f'No data for {no_data}')

No data for []

In [4]:
dfs["SOL"].iloc[-1, :]["tic"]

1712534400000

In [5]:
from collections import Counter


def get_majority_shape(dfs):
    # Get the shapes of all DataFrames
    shapes = [df.shape for df in dfs.values()]
    
    # Count occurrences of each shape
    shape_counts = Counter(shapes)
    
    # Display votes for each shape
    for shape, count in shape_counts.items():
        print(f"Shape: {shape}, Votes: {count}")
    
    # Find the most common shape
    majority_shape, majority_votes = shape_counts.most_common(1)[0]
    
    return majority_shape

def get_majority_start_end(dfs):
    # Get the shapes of all DataFrames
    start_end = [(df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"] ) for df in dfs.values()]
    
    # Count occurrences of each shape
    start_end_counts = Counter(start_end)
    
    # Display votes for each shape
    for se, count in start_end_counts.items():
        print(f"Start End: {se}, Votes: {count}")
    
    # Find the most common shape
    majority_se, majority_votes = start_end_counts.most_common(1)[0]
    
    return majority_se

major_shape = get_majority_shape(dfs)
major_se = get_majority_start_end(dfs)

Shape: (2881, 11), Votes: 56
Shape: (522, 11), Votes: 1
Shape: (1729, 11), Votes: 1
Shape: (2236, 11), Votes: 1
Shape: (2597, 11), Votes: 1
Start End: (1709942400000, 1712534400000), Votes: 56
Start End: (1712065500000, 1712534400000), Votes: 1
Start End: (1710979200000, 1712534400000), Votes: 1
Start End: (1710522900000, 1712534400000), Votes: 1
Start End: (1710198000000, 1712534400000), Votes: 1


In [6]:
def filter_dfs(dfs, shape, start_end):
    valid_dfs = {}
    for coin, df in dfs.items():
        if df.shape != shape:
            print(f"Shape mismatch for {coin}. Expected {shape}, got {df.shape}")
            continue
        elif (df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"]) != start_end:
            print(f"Start/End mismatch for {coin}. Expected {start_end}, got {df.iloc[0, :]['tic'], df.iloc[-1, :]['tic']}")
            continue
        else:
            valid_dfs[coin] = df

    return valid_dfs


valid_dfs = filter_dfs(dfs, major_shape, major_se)

Shape mismatch for ENA. Expected (2881, 11), got (522, 11)
Shape mismatch for ETHFI. Expected (2881, 11), got (1729, 11)
Shape mismatch for BOME. Expected (2881, 11), got (2236, 11)
Shape mismatch for kFLOKI. Expected (2881, 11), got (2597, 11)


In [7]:
df = valid_dfs["SOL"]

df.head()

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,
SOL,2024-03-09-00:00:00,2024-03-09-00:14:59,15m,436,146.12,146.39,144.84,145.09,9451.39,1709942400000,1709943299999
SOL,2024-03-09-00:15:00,2024-03-09-00:29:59,15m,399,146.24,146.31,145.72,146.10,8242.40,1709943300000,1709944199999
SOL,2024-03-09-00:30:00,2024-03-09-00:44:59,15m,402,146.54,146.59,145.18,146.20,8876.19,1709944200000,1709945099999
SOL,2024-03-09-00:45:00,2024-03-09-00:59:59,15m,286,146.40,147.20,146.27,146.50,7156.97,1709945100000,1709945999999
SOL,2024-03-09-01:00:00,2024-03-09-01:14:59,15m,421,144.95,146.42,144.95,146.41,24934.58,1709946000000,1709946899999


In [8]:
_df = pd.concat(valid_dfs.values())
_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 161336 entries, BTC to FET
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   start_time        161336 non-null  object 
 1   end_time          161336 non-null  object 
 2   interval          161336 non-null  object 
 3   number_of_trades  161336 non-null  int64  
 4   close             161336 non-null  float64
 5   high              161336 non-null  float64
 6   low               161336 non-null  float64
 7   open              161336 non-null  float64
 8   volume            161336 non-null  float64
 9   tic               161336 non-null  int64  
 10  toc               161336 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 14.8+ MB


In [9]:
_df.head()

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,
BTC,2024-03-09-00:00:00,2024-03-09-00:14:59,15m,450,68290.0,68340.0,68097.0,68201.0,31.80698,1709942400000,1709943299999
BTC,2024-03-09-00:15:00,2024-03-09-00:29:59,15m,454,68157.0,68291.0,68157.0,68291.0,41.31060,1709943300000,1709944199999
BTC,2024-03-09-00:30:00,2024-03-09-00:44:59,15m,450,68156.0,68234.0,68088.0,68151.0,25.44384,1709944200000,1709945099999
BTC,2024-03-09-00:45:00,2024-03-09-00:59:59,15m,439,68163.0,68247.0,68079.0,68161.0,25.34887,1709945100000,1709945999999
BTC,2024-03-09-01:00:00,2024-03-09-01:14:59,15m,512,67984.0,68167.0,67984.0,68163.0,20.26031,1709946000000,1709946899999


In [26]:
# import numpy as np


# df = _df
# ############ pct_change
# df["pct_change"] = df.groupby(level=0)["close"].pct_change() * 100

# ############ accumulated_pct_change
# df['multiplier'] = 1 + (df['pct_change'] / 100)
# df['accumulated_multiplier'] = df.groupby(level=0)['multiplier'].cumprod()
# df['accumulated_pct_change'] = (df['accumulated_multiplier'] - 1) * 100
# df.drop(columns=['multiplier', 'accumulated_multiplier'], inplace=True)

# ############ rolling_accumulated_pct_change
# df['start_time'] = pd.to_datetime(df['start_time'])
# df['multiplier'] = 1 + (df['pct_change'] / 100)
# window_size = 7 * 24 * 4  # Adjust this based on your exact data frequency
# df['rolling_accumulated_multiplier'] = df.groupby(level=0)['multiplier'].transform(
#     lambda x: x.rolling(window=window_size, min_periods=1).apply(np.prod)
# )
# df['rolling_accumulated_pct_change'] = (df['rolling_accumulated_multiplier'] - 1) * 100
# df.drop(columns=['multiplier', 'rolling_accumulated_multiplier'], inplace=True)



# df

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc,pct_change,accumulated_pct_change,rolling_accumulated_pct_change
symbol,,,,,,,,,,,,,,
BTC,2024-02-10 00:00:00,2024-02-10-00:14:59,15m,1258,47244.0000,47278.0000,47160.0000,47166.0000,24.81303,1707523200000,1707524099999,NaN,NaN,NaN
BTC,2024-02-10 00:15:00,2024-02-10-00:29:59,15m,1099,47204.0000,47283.0000,47186.0000,47244.0000,23.62129,1707524100000,1707524999999,-0.084667,-0.084667,-0.084667
BTC,2024-02-10 00:30:00,2024-02-10-00:44:59,15m,1429,47147.0000,47232.0000,47077.0000,47204.0000,39.37283,1707525000000,1707525899999,-0.120752,-0.205317,-0.205317
BTC,2024-02-10 00:45:00,2024-02-10-00:59:59,15m,1048,47116.0000,47153.0000,47079.0000,47146.0000,28.94293,1707525900000,1707526799999,-0.065752,-0.270934,-0.270934
BTC,2024-02-10 01:00:00,2024-02-10-01:14:59,15m,993,47110.0000,47158.0000,47082.0000,47122.0000,28.31493,1707526800000,1707527699999,-0.012735,-0.283634,-0.283634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LDO,2024-03-10 23:00:00,2024-03-10-23:14:59,15m,51,3.2533,3.2542,3.2205,3.2205,9344.20000,1710111600000,1710112499999,1.040437,8.671544,-2.560800
LDO,2024-03-10 23:15:00,2024-03-10-23:29:59,15m,20,3.2629,3.2629,3.2448,3.2484,6651.10000,1710112500000,1710113399999,0.295085,8.992217,-2.073830
LDO,2024-03-10 23:30:00,2024-03-10-23:44:59,15m,22,3.2740,3.2740,3.2563,3.2665,1128.30000,1710113400000,1710114299999,0.340188,9.362996,-1.604857


In [10]:
df.to_csv(DATA_DIR / "all.csv", index=True)